In [ ]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [ ]:
import pandas as pd
# from sklearn.model_selection import KFold

In [ ]:
df = pd.DataFrame(columns=["Label","Posts"])

In [ ]:
chunksize = 5000
for chunk in pd.read_csv("/content/drive/MyDrive/Thesis Papers/SMHD/final_cleaned_data.csv", chunksize=chunksize):
  df=df.append(chunk,ignore_index=True)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df["Label"].value_counts()

control          26602
depression       21830
bipolar          10005
ptsd              4471
schizophrenia     2092
Name: Label, dtype: int64

In [ ]:
df_c = df[df["Label"] == "control"]
df_d = df[df["Label"] == "ptsd"]

df_c = df_c.sample(frac = 1)
df_c = df_c.sample(frac = 1).iloc[0:3000,]  #control 3k
df_d = df_d.sample(frac = 1)  # 4471 ptsd
# df_d = df_d.sample(frac = 1).iloc[0:5000,]

# df_c = df_c.iloc[0:5000,]
# df_d = df_d.iloc[0:5000,] 

# 

In [ ]:
def stripstr(x):
  return len(x.strip())==0

In [ ]:
result = pd.concat([df_c, df_d], ignore_index=True, sort=False)
empty = result[result["Posts"].apply(stripstr)]
indexarr = empty.index
result.drop(index = indexarr, inplace=True)

In [ ]:
result = result.sample(frac = 1)

In [ ]:
import re
import pandas as pd

import nltk
nltk.download('stopwords')

stopwords = nltk.corpus.stopwords.words('english')
wn = nltk.WordNetLemmatizer()
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [ ]:
def tokenize(txt):
  tokens = re.split('\W+',txt) # W means non-word characters and + means one or more ["hello", 'my']
  return tokens

def lemmed(txt):
  text = [wn.lemmatize(word) for word in txt]
  return text

def remove_stopwords(txt):
  clean_msg = [word for word in txt if word not in stopwords]
  return clean_msg

In [ ]:
result.describe()

,Label,Posts
count,7447,7447
unique,2,5733
top,ptsd,thank you much appreciated d yesssssss hey e...
freq,4471,2


In [ ]:
result['Posts'] = result['Posts'].apply(lambda x: tokenize(x))
result['Posts'] = result['Posts'].apply(remove_stopwords)
result['Posts'] = result['Posts'].apply(lemmed)
result["Posts"]=result["Posts"].apply(lambda x: " ".join(x))

In [ ]:
result

,Label,Posts
1224,control,get every always take next level month break ...
6456,ptsd,thanks shout original poster haha sorry long w...
4435,ptsd,really happen really shocking thing creepy mo...
3466,ptsd,hmmsounds super shady therapist would recomme...
7037,ptsd,first close heart attack read thing honestly e...
...,...,...
1453,control,whenever attempt made record paranormal activi...
6958,ptsd,chrissy marry fuck kill mario kart super mario...
3730,ptsd,hear fine thank feeling aba therapy isnt liter...
5388,ptsd,thank everyone feedback appreciate wasnt hars...


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

In [ ]:
param_grid = {
    "C" : [10,80],
    "gamma" : [1, 0.01],
    "kernel" : ["rbf", "poly"]
}

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
cv = TfidfVectorizer(max_features=4000)
X = cv.fit_transform(result["Posts"]).toarray() # all the contents

In [ ]:
y=result["Label"]    #all the labels 

In [ ]:
Y = y.to_numpy() #numpy array representation of labels

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(X,Y,test_size=0.3,random_state=42)     # x_train is 70% trainig dta, y_train 70% t-label, 

In [ ]:
# def cut(sen, length = 3000):
#   content = sen.split()
#   if len(content) <= length:
#     return " ".join(i for i in content if i not in stop)

#   return " ".join(i for i in content[0:length] if i not in stop)

# filter1=df["Label"].isin(["control","depression"])
# df_cd=df[filter1]

# SVM

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn import metrics

In [ ]:
model = SVC(C=80, gamma = 0.01, kernel = 'rbf')# c=50, g=0.001, kernel=rbf pre=0.9

In [ ]:
model.fit(x_train, y_train)

SVC(C=80, gamma=0.01)

In [ ]:
predictions = model.predict(x_test)

In [ ]:
print(metrics.classification_report(y_test,predictions))

              precision    recall  f1-score   support

     control       0.86      0.87      0.87       930
        ptsd       0.91      0.90      0.90      1305

    accuracy                           0.89      2235
   macro avg       0.89      0.89      0.89      2235
weighted avg       0.89      0.89      0.89      2235



In [ ]:
grid = GridSearchCV(model, param_grid, refit = True)

In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [ ]:
grid.fit(x_train,y_train)

In [ ]:
# text_clf=Pipeline([("tfidf", TfidfVectorizer()),("clf", LinearSVC(random_state=0, tol=1e-5))])
# text_clf.fit(x_train,y_train)

In [ ]:
predictions = text_clf.predict(x_test)

In [ ]:
print(metrics.confusion_matrix(y_test,predictions))

[[1241  271]
 [ 322 1166]]


In [ ]:
print(metrics.classification_report(y_test,predictions))

              precision    recall  f1-score   support

     control       0.79      0.82      0.81      1512
  depression       0.81      0.78      0.80      1488

    accuracy                           0.80      3000
   macro avg       0.80      0.80      0.80      3000
weighted avg       0.80      0.80      0.80      3000



In [ ]:
print(metrics.accuracy_score(y_test,predictions))

0.8023333333333333
